# Frequency Graph

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set paths for matrices from TSV files
# First column will be index of column which will be used to identify each row
plasmid_matrix = pd.read_csv('path_to_plasmid_frequency_matrix', sep='\t', index_col=0)
mutating_cells_matrix = pd.read_csv('path_to_mutating_frequency_matirx', sep='\t', index_col=0)
treated_cells_matrix = pd.read_csv('path_to_drug_treated_frequency_matrix', sep='\t', index_col=0)

# Extract the numbers from the index column
# These numbers are stored in a new column called position 
plasmid_matrix['Position'] = plasmid_matrix.index.str.extract(r'(\d+)', expand=False).astype(int)
mutating_cells_matrix['Position'] = mutating_cells_matrix.index.str.extract(r'(\d+)', expand=False).astype(int)
treated_cells_matrix['Position'] = treated_cells_matrix.index.str.extract(r'(\d+)', expand=False).astype(int)

# Sort by position column
plasmid_matrix.sort_values('Position', inplace=True)
mutating_cells_matrix.sort_values('Position', inplace=True)
treated_cells_matrix.sort_values('Position', inplace=True)

# Function to plot frequency data with logarithmic scale
def plot_frequencies_log_with_black_fill(matrix, title):
    plt.figure(figsize=(12, 6))
    # Exclude the position column
    for sample in matrix.columns[:-1]: 
        plt.plot(matrix['Position'], matrix[sample], marker='None', linestyle='-', color='black', linewidth=1)
        # The plot is filled black
        plt.fill_between(matrix['Position'], matrix[sample], color='black', alpha=1.0)  

    plt.xlabel('Position')
    plt.ylabel('Frequency')
    # y-axis set to logarithmic scale
    plt.yscale('log')  
    plt.ylim(1e-6, 0.1)  
    plt.title(title)
    plt.show()

# Plotting the frequency graphs
plot_frequencies_log_with_black_fill(plasmid_matrix, 'Logarithmic Frequencies of Mutations in Plasmid Samples')
plot_frequencies_log_with_black_fill(mutating_cells_matrix, 'Logarithmic Frequencies of Mutations in Mutating Cells Samples')
plot_frequencies_log_with_black_fill(treated_cells_matrix, 'Logarithmic Frequencies of Mutations in Treated Cells Samples')


# Enrichment graph

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set path for enrichment tsv file
file_path = 'path_to_enrichment_file'
data = pd.read_csv(file_path, sep='\t')

# Convert all values to numeric
# Non-numeric set as missing values
data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# 999999 values replaced with NaN temporarily
data.replace(999999, pd.NA, inplace=True)

# Find the maximum value below 999999 from all columns
global_max_value = data.iloc[:, 1:].max().max(skipna=True)

# Cap NaN values (that were originally 999999) to slightly above the max non-NaN number value
value_increment = 0.01  
# Replace all NaN values with global_max_value + value_increment
data_capped = data.iloc[:, 1:].applymap(lambda x: global_max_value + value_increment if pd.isna(x) else x)

# Function to extract position from amino acid mutation 
def extract_position(mutation_str):
    position = ''
    started = False
    for char in mutation_str:
        if char.isdigit():
            started = True
            position += char
        elif started:
            break
    return int(position) if position else None

# Add the position column to the dataframe
data['position'] = data.iloc[:, 0].apply(extract_position)

# Drop any rows where position is missing or missing rows
data.dropna(subset=['position'], inplace=True)

# Set the position column as the index
data.set_index('position', inplace=True)

# Create the three graphs 
comparison_groups = {
    "Plasmid vs Mutating Cells": ["13vs17", "14vs18", "15vs19", "16vs20"],
    "Plasmid vs Drug-Treated": ["13vs21", "14vs22", "15vs23", "16vs24"],
    "Mutating Cells vs Drug-Treated": ["17vs21", "18vs22", "19vs23", "20vs24"]
}

for title, columns in comparison_groups.items():
    plt.figure(figsize=(10, 6))
    for column in columns:
        plt.plot(data.index, data_capped[column], color='black')
        # Plot filled black
        plt.fill_between(data.index, data_capped[column], color='black', alpha=1.0)  
    plt.xlabel('Position')
    plt.ylabel('Enrichment Value')
    plt.title(title)
    # Set y-axis limit
    plt.ylim(0, global_max_value + value_increment + 10)  
    plt.show()

# Pie chart

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Load enrichment tsv file
data_path = 'path_to_enrichment_file'
data = pd.read_csv(data_path, sep='\t')

# Set column names manually 
columns = ['position_mutation', '13vs17', '14vs18', '15vs19', '16vs20', '13vs21', '14vs22', '15vs23', '16vs24', '17vs21', '18vs22', '19vs23', '20vs24']
data.columns = columns

# Makes sure position_mutation is read as string
data['position_mutation'] = data['position_mutation'].astype(str)

# Extract numberic from position_mutation column
# Extract mutation from and puts it in mutation column
data['position'] = data['position_mutation'].str.extract(r'(\d+)').astype(int)
data['mutation'] = data['position_mutation'].str.extract(r'([A-Z]+\d+.*)')

# Change the dataframe to long format
data_melted = data.melt(id_vars=['position', 'mutation'], 
                        var_name='comparison', 
                        value_name='enrichment')

# Convert position column to numeric 
data_melted['position'] = pd.to_numeric(data_melted['position'])

# Convert enrichment to numeric
data_melted['enrichment'] = pd.to_numeric(data_melted['enrichment'], errors='coerce')


# Remove rows with null enrichment values
data_melted = data_melted.dropna(subset=['enrichment'])

# Filter to include only enrichments 2 or above
data_melted = data_melted[data_melted['enrichment'] >= 2]

# Handle multiple enrichment values by taking the maximum
data_melted = data_melted.sort_values(by=['position', 'mutation', 'enrichment'], ascending=[True, True, False])
data_melted = data_melted.drop_duplicates(subset=['position', 'mutation'], keep='first')

# Define the comparison groups
comparison_groups = {
    'Plasmid vs Mutating Cells': ["13vs17", "14vs18", "15vs19", "16vs20"],
    'Plasmid vs Drug-Treated': ["13vs21", "14vs22", "15vs23", "16vs24"],
    'Mutating Cells vs Drug-Treated': ["17vs21", "18vs22", "19vs23", "20vs24"]
}

# Assigns comparison to each group
data_melted['comparison_group'] = data_melted['comparison'].apply(
    lambda x: next((k for k, v in comparison_groups.items() if x in v), 'Other')
)

# Count the occurrences of each comparison group 
comparison_group_counts = data_melted['comparison_group'].value_counts().reset_index()
comparison_group_counts.columns = ['comparison_group', 'count']

# Create a pie chart 
fig, ax = plt.subplots()
colors = ['#FF6347', '#4682B4', '#32CD32']  
wedges, texts, autotexts = ax.pie(comparison_group_counts['count'], 
                                  autopct=lambda p: '{:.0f}'.format(p * sum(comparison_group_counts['count']) / 100), 
                                  startangle=90, 
                                  colors=colors)

ax.axis('equal')  

# Title
plt.title('Number of Enrichments in Different Conditions (Enrichments >= 2)')

# Legend
ax.legend(wedges, comparison_group_counts['comparison_group'], title="Comparison Group", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

# Remove text labels from around the pie chart, to only keep the actual counts
for text in texts:
    text.set_text('')

for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontsize(10)

plt.show()

# set Output directory 
output_dir = 'set_directory'
os.makedirs(output_dir, exist_ok=True)

# Save plot
output_file = os.path.join(output_dir, 'pie_chart_enrichments.png')
fig.savefig(output_file, bbox_inches='tight')

# violin plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load enrichment tsv file
file_path = 'path_to_enrichment_file'
data = pd.read_csv(file_path, sep='\t')

# Convert all values to numeric
# Non-numeric set as missing values
data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# 999999 values replaced with NaN temporarily
data.replace(999999, pd.NA, inplace=True)

# Find the maximum value below 999999 from all columns
global_max_value = data.iloc[:, 1:].max().max(skipna=True)

# Cap NaN values (that were originally 999999) to slightly above the max non-NaN number value
value_increment = 0.01  
# Replace all NaN values with global_max_value + value_increment
data_capped = data.iloc[:, 1:].applymap(lambda x: global_max_value + value_increment if pd.isna(x) else x)

# Function to extract position from amino acid mutation 
def extract_position(mutation_str):
    position = ''
    started = False
    for char in mutation_str:
        if char.isdigit():
            started = True
            position += char
        elif started:
            break
    return int(position) if position else None

# Add position column to the dataframe
data['position'] = data.iloc[:, 0].apply(extract_position)

# Drop any rows where position is missing or missing rows
data.dropna(subset=['position'], inplace=True)

# Set the position column as the index
data.set_index('position', inplace=True)

# Assign comparison groups
comparison_groups = {
    "Plasmid vs Mutating Cells": ["13vs17", "14vs18", "15vs19", "16vs20"],
    "Plasmid vs Drug-Treated": ["13vs21", "14vs22", "15vs23", "16vs24"],
    "Mutating Cells vs Drug-Treated": ["17vs21", "18vs22", "19vs23", "20vs24"]
}

long_df_list = []

for group_name, columns in comparison_groups.items():
    melted_df = data_capped[columns].melt(var_name='Sample', value_name='Enrichment', ignore_index=False)
    melted_df['Comparison'] = group_name
    long_df_list.append(melted_df)

combined_enrichment_df = pd.concat(long_df_list).reset_index()

# Plotting the violin plot  
plt.figure(figsize=(12, 6))
sns.violinplot(x='Comparison', y='Enrichment', data=combined_enrichment_df, inner='quartile', scale='width', palette='Set2')
plt.title('Distribution of Enrichment Values For Amino Acids')
plt.xlabel('Comparison')
plt.ylabel('Enrichment')
plt.xticks(rotation=360)  

# Enrichment map

In [ ]:
import os
import pandas as pd
import altair as alt

# Remove the limit of ~5000 rows, to handle larger dataset
alt.data_transformers.disable_max_rows()

# Load enrichment tsv file
file_path = 'path_to_enrichment_file'
data = pd.read_csv(file_path, sep='\t')

# Set column names manually
columns = ['position_mutation', '13vs17', '14vs18', '15vs19', '16vs20', '13vs21', '14vs22', '15vs23', '16vs24', '17vs21', '18vs22', '19vs23', '20vs24']
data.columns = columns

# Ensure position_mutation is read as string
data['position_mutation'] = data['position_mutation'].astype(str)

# Function to extract position from amino acid mutation 
def extract_position(mutation_str):
    for i, char in enumerate(mutation_str):
        if char.isdigit():
            position = ''
            while i < len(mutation_str) and mutation_str[i].isdigit():
                position += mutation_str[i]
                i += 1
            return position
    return None

# Function to extract the mutation from amino acid mutation 
def extract_mutation(mutation_str):
    for i, char in enumerate(mutation_str):
        if char.isdigit():
            while i < len(mutation_str) and mutation_str[i].isdigit():
                i += 1
            return mutation_str[i:]
    return None

# Function to categorise mutations
def categorize_mutation(mutation):
    if 'fs' in mutation:
        return 'Frameshift'
    elif 'insdel' in mutation:
        return 'Insdel'
    elif 'ins' in mutation:
        return 'Insertion'
    elif 'del' in mutation:
        return 'Deletion'
    elif mutation.endswith('X'):
        return 'Nonsense'
    elif mutation.isupper():
        return mutation  # This will capture specific missense mutations as their new amino acid
    else:
        return 'Other'

# Split 'position_mutation' into separate 'position' and 'mutation' columns
data['position'] = data['position_mutation'].apply(lambda x: extract_position(x))
data['mutation'] = data['position_mutation'].apply(lambda x: extract_mutation(x))

# Drop the original 'position_mutation' column
data = data.drop(columns=['position_mutation'])

# Drop any rows with NaN positions
data = data.dropna(subset=['position'])

# Convert 'position' to numeric 
data['position'] = pd.to_numeric(data['position'], errors='coerce')

# Categorise mutations
data['mutation_category'] = data['mutation'].apply(categorize_mutation)

# selecting columns of interest. Comment out the comparisons you dont want
# Select only the columns for plasmid vs mutating cells
plasmid_vs_mutating_columns = ["13vs17", "14vs18", "15vs19", "16vs20"]
data = data[['position', 'mutation_category'] + plasmid_vs_mutating_columns]

# Select only the columns for plasmid vs drug-treated cells
#plasmid_vs_drugtreated_columns = ["13vs21", "14vs22", "15vs23", "16vs24"]
#data = data[['position', 'mutation_category'] + plasmid_vs_drugtreated_columns]

# Select only the columns for mutating vs drug-treated cells
#mutating_vs_drugtreated_columns = ["17vs21", "18vs22", "19vs23", "20vs24"]
#data = data[['position', 'mutation_category'] + mutating_vs_drugtreated_columns]


data_melted = data.melt(id_vars=['position', 'mutation_category'], 
                        var_name='comparison', 
                        value_name='enrichment')

# Ensure enrichment read as numeric
data_melted['enrichment'] = pd.to_numeric(data_melted['enrichment'], errors='coerce')

# Cap the enrichment values at 100
data_melted['enrichment'] = data_melted['enrichment'].apply(lambda x: 100 if x == 999999 else x)

# Remove any rows with null, zero, or less than 2 enrichment values
data_melted = data_melted.dropna(subset=['enrichment'])
data_melted = data_melted[data_melted['enrichment'] >= 2]

# Handle multiple enrichment values by taking the maximum 
data_melted = data_melted.sort_values(by=['position', 'mutation_category', 'enrichment'], ascending=[True, True, False])
data_melted = data_melted.drop_duplicates(subset=['position', 'mutation_category'], keep='first')

# Define order of mutation categories on y-axis
base_categories = ['Frameshift', 'Insdel', 'Insertion', 'Deletion', 'Nonsense', 'Other']
specific_missense_mutations = sorted([cat for cat in data_melted['mutation_category'].unique() if cat not in base_categories])
mutation_category_order = base_categories + specific_missense_mutations


data_melted['mutation_category'] = pd.Categorical(data_melted['mutation_category'], categories=mutation_category_order, ordered=True)

# Define plot parameters
cell_size = 15  
max_width = 1750  
max_height = 600  

width = min(cell_size * data_melted['position'].nunique(), max_width)
height = min(cell_size * len(mutation_category_order), max_height)

# Set the x-axis range explicitly from 25 to 247
x_min = 25
x_max = 247
x_step = 5

# Generate x-axis values from 25 in steps of 10
x_axis_values = list(range(x_min, x_max + 1, x_step))

# Tooltips
heatmap_tooltips = [
    alt.Tooltip('position:O', title="Position"),
    alt.Tooltip('mutation_category:N', title="Mutation Category"),
    alt.Tooltip('enrichment:Q', title="Enrichment", format=".2f"),
    alt.Tooltip('comparison:N', title="File Comparison")
]

# Highlight cells when hovered over
amino_acid_selection = alt.selection_point(encodings=['x', 'y'], on='mouseover', empty=False)

# Define heatmap function with custom tick values
def heatmap(data, title):
    base = alt.Chart(data).encode(
        x=alt.X('position:O',
                axis=alt.Axis(
                    titleFontSize=15, 
                    labelFontSize=12, 
                    values=x_axis_values, 
                    tickCount=len(x_axis_values)  
                )),
        y=alt.Y('mutation_category:O',
                sort=mutation_category_order,
                axis=alt.Axis(
                    labelFontSize=12,  
                    titleFontSize=15, 
                    orient='left'
                ))
    ).properties(height=height, width=width, title=title)
    
    heatmap = base.mark_rect().encode(
        color=alt.value('blue'),  
        stroke=alt.value('black'),
        strokeWidth=alt.condition(amino_acid_selection, alt.value(2), alt.value(0)),
        tooltip=heatmap_tooltips
    )
    
    return heatmap.add_params(amino_acid_selection)

# Create heatmap
final_plot = heatmap(data_melted, "Plasmid vs Mutating")

# set Output directory 
output_dir = 'set_directory'
os.makedirs(output_dir, exist_ok=True)

# Save the plot to HTML
output_html = os.path.join(output_dir, 'heatmap_multi_plasmidvsmutating.html')
final_plot.save(output_html)

# Display plot
final_plot